# Parte 1: Montar la base de datos.

In [21]:
# Montar Google Drive (esto montará todo tu Drive)
from google.colab import drive
drive.mount('/content/drive')

# Instalar librerías necesarias (ejecutar solo una vez)
!pip install pandas matplotlib seaborn openpyxl gspread google-auth

print("Dependencias instaladas y Drive montado.")

# Ahora puedes acceder a tu archivo de Sheets
# Primero, necesitas la ID de tu Google Sheet (está en la URL)
# URL: https://docs.google.com/spreadsheets/d/1DgVvDZp_UR9o_0drm9X59i41x2Fg8SLk/edit...
SHEET_ID = '1DgVvDZp_UR9o_0drm9X59i41x2Fg8SLk'

# Opción A: Si el archivo está en tu Drive, puedes exportarlo como CSV
import pandas as pd

url = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/export?format=csv'
df = pd.read_csv(url)

print("Datos cargados exitosamente:")
print(df.head())

Mounted at /content/drive
Dependencias instaladas y Drive montado.
Datos cargados exitosamente:
   Product_ID Production_Line    Shift  Units_Produced  Defect_Rate  \
0           1          Line_A  Morning             253        32.00   
1           2          Line_A  Morning             234        35.00   
2           3          Line_A  Morning             196        34.00   
3           4          Line_A  Morning             237        24.00   
4           5          Line_A  Evening             232         0.03   

   Energy_Consumption_kWh  Worker_Count  Downtime_Minutes  Temperature_C  \
0                   120.2            22                22           25.9   
1                   108.0            24                29           26.2   
2                    88.0            21                34           23.7   
3                   112.8            20                24           25.4   
4                   115.3            23                30           28.2   

   Pressure_Bar  Cos

# Parte 2: Cargar y procesar la información

In [23]:
import pandas as pd
import re

# --- CONFIGURACION: CAMBIA ESTA RUTA ---
# Reemplaza con la ruta correcta dentro de tu Google Drive
# Ejemplo: '/content/drive/MyDrive/Colab Notebooks/industry_dataset_structured.xlsx.pdf'
pdf_path = '/content/drive/MyDrive/industry_dataset_structured.xlsx.pdf' # <--- MODIFICA ESTO
# ----------------------------------------

def parse_industry_pdf(filepath):
    """Lee el PDF como texto y lo convierte en un DataFrame estructurado."""
    try:
        # Intentar leer el PDF como texto
        with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
            lines = f.readlines()
    except FileNotFoundError:
        print("Error: Archivo no encontrado en", filepath, "Revisa la ruta.")
        return None
    except Exception as e:
        print("Error al leer el archivo:", e)
        print("Por favor, asegurate de que el archivo sea un PDF con texto seleccionable.")
        return None

    # Convertir lineas a un string gigante
    full_text = " ".join(lines)

    # --- Extraccion de datos (Regex) ---
    # Patron mejorado para capturar las lineas de datos
    # Formato: Numero, Linea, Turno, Units, Downtime, Energy, Temp, Defect, Cost, Revenue, Batch
    pattern = r"(\d+)\s+(Line_[ABC])\s+(Morning|Evening|Night)\s+(\d+)\s+(\d+\.?\d*)\s+(\d+\.?\d*)\s+(\d+)\s+(\d+)\s+(\d+\.?\d*)\s+(\d+\.?\d*)\s+(\d+\.?\d*)\s+(\d+)"

    matches = re.findall(pattern, full_text)

    if not matches:
        print("No se encontraron datos con el patron. Revisa el formato del archivo.")
        # Imprimir un poco del texto para depurar
        print("\n--- Primeros 500 caracteres del archivo ---")
        print(full_text[:500])
        return None

    print("Se encontraron", len(matches), "registros.")

    # Crear DataFrame
    df = pd.DataFrame(matches, columns=['Product_ID', 'Production_Line', 'Shift', 'Units_Produced', 'Downtime_Minutes',
                                            'Energy_Consumption_kWh', 'Temperature_C', 'Operator_Count', 'Defect_Rate',
                                            'Cost_per_Unit', 'Revenue_USD', 'Batch'])

    # Convertir tipos de datos
    numeric_cols = ['Units_Produced', 'Downtime_Minutes', 'Energy_Consumption_kWh', 'Temperature_C',
                    'Operator_Count', 'Defect_Rate', 'Cost_per_Unit', 'Revenue_USD', 'Batch']

    for col in numeric_cols:
        # Algunos numeros pueden tener '0.0X' como string, convertimos a float primero
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Asegurar que Product_ID sea entero
    df['Product_ID'] = pd.to_numeric(df['Product_ID'], errors='coerce').astype('Int64')

    print("Datos procesados correctamente.")
    print("Dimensiones del DataFrame:", df.shape)
    return df

# Cargar los datos
df = parse_industry_pdf(pdf_path)

if df is not None:
    print("\n--- Primeras 5 filas del DataFrame ---")
    print(df.head())
    print("\n--- Informacion del DataFrame ---")
    print(df.info())
    print("\n--- Estadisticas descriptivas ---")
    print(df.describe())
else:
    print("No se pudo cargar el DataFrame. No se pueden generar las graficas.")

Error: Archivo no encontrado en /content/drive/MyDrive/industry_dataset_structured.xlsx.pdf Revisa la ruta.
No se pudo cargar el DataFrame. No se pueden generar las graficas.


# Parte 3: Configuracion global de estilo y unidades

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats
import matplotlib.ticker as mtick

# --- Verificar que df existe ---
if df is None:
    raise Exception("El DataFrame 'df' no esta cargado. Ejecuta la Parte 2 primero.")

# --- Configuracion de estilo global ---
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.linestyle'] = '--'
plt.rcParams['grid.linewidth'] = 0.7
plt.rcParams['grid.alpha'] = 0.6
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.left'] = True
plt.rcParams['axes.spines.bottom'] = True
plt.rcParams['axes.edgecolor'] = '#222222'
plt.rcParams['axes.linewidth'] = 1.2
plt.rcParams['legend.frameon'] = False

# --- Colores y formas consistentes ---
line_colors = {'Line_A': '#1f77b4', 'Line_B': '#ff7f0e', 'Line_C': '#2ca02c'}
line_markers = {'Line_A': 'o', 'Line_B': 's', 'Line_C': '^'}
shift_styles = {'Morning': {'color': '#9467bd', 'linestyle': '-'},
                'Evening': {'color': '#8c564b', 'linestyle': '--'},
                'Night': {'color': '#d62728', 'linestyle': '-.'}}

# Colores para turnos (para graficos de barras, etc.)
shift_color_map = {'Morning': '#9467bd', 'Evening': '#8c564b', 'Night': '#d62728'}

# --- Funciones auxiliares ---
def error_std(data):
    """Calcula la media y la desviacion estandar para barras de error."""
    mean = data.mean()
    std = data.std()
    return mean, std

def error_ee(data):
    """Calcula la media y el error estandar (EE) para barras de error."""
    mean = data.mean()
    ee = data.std() / np.sqrt(len(data))
    return mean, ee

print("Estilos y utilidades configurados.")
print("Colores de lineas:", line_colors)
print("Colores de turnos:", shift_color_map)

Exception: El DataFrame 'df' no esta cargado. Ejecuta la Parte 2 primero.

In [ ]:
import os

# List contents of your Google Drive 'MyDrive' folder
drive_path = '/content/drive/MyDrive'

print(f"Contenido de {drive_path}:")
if os.path.exists(drive_path):
    for root, dirs, files in os.walk(drive_path):
        for name in files:
            print(os.path.join(root, name))
        for name in dirs:
            print(os.path.join(root, name))
else:
    print(f"El directorio {drive_path} no existe. Asegurate de que Google Drive este montado correctamente.")

Contenido de /content/drive/MyDrive:
/content/drive/MyDrive/rwservlet (1).pdf
/content/drive/MyDrive/MMUL4N (2020) Telegram @PelisDrive.mkv
/content/drive/MyDrive/Copia de MMUL4N (2020) Telegram @PelisDrive.mkv
/content/drive/MyDrive/Citar bibliografía en estilo APA.pdf
/content/drive/MyDrive/Escaneado_20210816-1426.pdf
/content/drive/MyDrive/Unit 1.pdf
/content/drive/MyDrive/ADOLFO HUERTA ROMO - Actividad preliminar..gdoc
/content/drive/MyDrive/MAPA CONCEPTUAL DEL CAPITULO I DE CISCO.pdf
/content/drive/MyDrive/actividad integradora.pdf
/content/drive/MyDrive/Adolfo Huerta_U1Act1.1.pdf
/content/drive/MyDrive/CAPITULO 2 DE CISCO.pdf
/content/drive/MyDrive/Ejemplos con la Sentencia de Asignación.pdf
/content/drive/MyDrive/Escaneado_20210901-1620.pdf
/content/drive/MyDrive/Adolfo Huerta_U1Act1.2.pdf
/content/drive/MyDrive/SEMANA 4. MAPA CONCEPTUAL CAPITULO 3.pdf
/content/drive/MyDrive/Tipos de Datos.pdf
/content/drive/MyDrive/Teorías del origen de nuestro mundo.pdf
/content/drive/MyDri

# Parte 4
## Figura 1: Depresion por linea

In [ ]:
# --- Fig. 1: Units Produced vs Defect Rate ---
plt.figure(figsize=(10, 6))

for line, group in df.groupby('Production_Line'):
    plt.scatter(group['Units_Produced'], group['Defect_Rate'],
                    color=line_colors[line], marker=line_markers[line],
                                    label=line, alpha=0.7, edgecolors='black', linewidth=0.5)

                                    plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(1.0)) # Defect_Rate es proporción (e.g., 0.04 = 4%)
                                    plt.xlabel('Units Produced')
                                    plt.ylabel('Defect Rate')
                                    plt.title('Fig. 1: Units Produced vs Defect Rate')
                                    plt.legend(title='Production_Line')
                                    plt.tight_layout()
                                    plt.show()

                                    # --- Interpretación (Escríbela basada en la gráfica) ---
                                    print("\n--- Interpretación Fig. 1 ---")
                                    print("Observa si existe una tendencia (positiva/negativa) o si los puntos están dispersos sin patrón.")
                                    print("Identifica si hay outliers (puntos muy alejados).")
                                    print("Por ejemplo: 'No se observa una correlación clara entre el volumen de producción y la tasa de defectos. Los puntos de las tres líneas se entremezclan, aunque Line_B (naranja) parece tener una tasa de defectos ligeramente más baja en general.'")

# Parte 5
## Figura 2 – Dispersion y tendencia OLS

In [ ]:
# --- Fig. 2: Energy_Consumption_kWh vs Revenue_USD ---
plt.figure(figsize=(10, 6))

x = df['Energy_Consumption_kWh']
y = df['Revenue_USD']

# Calcular regresión lineal
slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
r_squared = r_value**2
corr_coef = r_value

plt.scatter(x, y, color='lightgreen', edgecolors='black', linewidth=0.5, alpha=0.7, label='Observaciones')
plt.plot(x, intercept + slope * x, '--', color='darkred', linewidth=2, label=f'Tendencia (OLS)')

plt.gca().yaxis.set_major_formatter(mtick.StrMethodFormatter('${x:,.0f}'))
plt.xlabel('Energy Consumption (kWh)')
plt.ylabel('Revenue (USD)')
plt.title(f'Fig. 2: Energía vs Ingresos (r = {corr_coef:.2f})')
plt.legend()
plt.tight_layout()
plt.show()

print("\n--- Interpretación Fig. 2 ---")
print(f"El coeficiente de correlación es r = {corr_coef:.2f}.")
print("Describe la dirección y fuerza de la relación. Por ejemplo: 'Existe una correlación positiva fuerte (r=0.XX), lo que sugiere que a mayor consumo de energía, mayores son los ingresos, probablemente debido a un mayor throughput.'")
print("Menciona si hay outliers evidentes.")

# Parte 6
## Figura 3 - Distribución de defect rate

In [ ]:
# --- Fig. 3: Distribución de Defect_Rate (Histograma) ---
plt.figure(figsize=(10, 6))

defect_rate = df['Defect_Rate'].dropna()

n_bins = 20
counts, bins, patches = plt.hist(defect_rate, bins=n_bins, color='lightblue',
                                 edgecolor='darkblue', linewidth=1.2, alpha=0.7)

                                 mean_val = defect_rate.mean()
                                 median_val = defect_rate.median()

                                 plt.axvline(mean_val, color='green', linestyle='--', linewidth=2, label=f'Media: {mean_val:.2%}')
                                 plt.axvline(median_val, color='red', linestyle='-.', linewidth=2, label=f'Mediana: {median_val:.2%}')

                                 plt.gca().xaxis.set_major_formatter(mtick.PercentFormatter(1.0))
                                 plt.xlabel('Defect Rate')
                                 plt.ylabel('Frecuencia')
                                 plt.title('Fig. 3: Distribución de Defect_Rate')
                                 plt.legend()
                                 plt.tight_layout()
                                 plt.show()

                                 print("\n--- Interpretación Fig. 3 ---")
                                 print("Indica si la distribución es simétrica o sesgada (a la izquierda/derecha).")
                                 print("¿Dónde se concentra la mayor parte de los datos?")
                                 print("Por ejemplo: 'La distribución está ligeramente sesgada a la derecha, con una media (X%) mayor que la mediana (Y%). La mayoría de las tasas de defecto se concentran entre el 4% y el 6%.'")